In [2]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import StepColormap

# Load cleaned + joined grocery store data
df = pd.read_csv("../data/cleaned/grocery_stores_cleaned_v4.csv")
print(f"✅ Loaded: {df.shape[0]} rows")

# Filter to only real grocery stores
real_grocery = df[df["IS_REAL_GROCERY"] == True]

# Count stores by community
grocery_counts = (
    real_grocery["Community"]
    .value_counts()
    .reset_index()
)
grocery_counts.columns = ["Community", "StoreCount"]
print("✅ Sample counts:\n", grocery_counts.head())

# Load Community Area GeoJSON
geo = gpd.read_file("../docs/community_areas.geojson")
geo = geo.rename(columns={"community": "Community"})  # Rename to match merge key
print("✅ Geo columns:", geo.columns.tolist())

# Merge counts into polygons
geo = geo.merge(grocery_counts, on="Community", how="left")
geo["StoreCount"] = geo["StoreCount"].fillna(0)

# Create map
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Ensure numeric StoreCount
geo["StoreCount"] = pd.to_numeric(geo["StoreCount"], errors="coerce").fillna(0).astype(float)
print(geo["StoreCount"].describe())

# Updated color scale: brighter yellow for 29-39
choropleth_scale = StepColormap(
    colors=[
        "#b30000",  # 0-10 = Very red
        "#f03b20",  # 10-20 = Strong orange
        "#fc9272",  # 20-29 = Light red
        "#ffff33",  # 29-39 = Bright yellow
        "#31a354"   # 39-49+ = Green
    ],
    index=[0, 10, 20, 29, 39, 50],  # Explicit cutoffs
    vmin=0,
    vmax=50,
    caption="Number of Real Grocery Stores per Community Area"
)

# Add GeoJSON Layer
folium.GeoJson(
    geo,
    name="Grocery Store Density",
    style_function=lambda feature: {
        "fillColor": choropleth_scale(feature["properties"]["StoreCount"]) if feature["properties"]["StoreCount"] > 0 else "#f0f0f0",
        "color": "white",
        "weight": 1,
        "fillOpacity": 0.7,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["Community", "StoreCount"],
        aliases=["Community Area", "# of Real Grocery Stores"],
        localize=True
    )
).add_to(m)

# Add color scale
choropleth_scale.add_to(m)

# Save map
output_path = "../docs/grocery_choropleth_by_area.html"
m.save(output_path)
print(f"✅ Choropleth map saved to: {output_path}")


✅ Loaded: 4504 rows
✅ Sample counts:
          Community  StoreCount
0    HUMBOLDT PARK          16
1  NEAR NORTH SIDE          15
2       WEST RIDGE          14
3   BELMONT CRAGIN          14
4     LOGAN SQUARE          13
✅ Geo columns: ['Community', 'shape_area', 'area_num_1', 'area_numbe', 'shape_len', 'geometry']
count    77.000000
mean      3.935065
std       4.046889
min       0.000000
25%       1.000000
50%       3.000000
75%       5.000000
max      16.000000
Name: StoreCount, dtype: float64
✅ Choropleth map saved to: ../docs/grocery_choropleth_by_area.html
